In [1]:
from __future__ import print_function

import sys
import numpy as np
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split

sys.path.append('..')
import tensorflow as tf
from supplementary_code_direct_ranker.DirectRanker import directRanker
from supplementary_code_direct_ranker.helpers import readData, nDCGScorer_cls, MAP_cls

# Read data

In [2]:
x_test, y_test, q_test = readData(data_path="../data/MSLR-WEB10K/Fold1/test.txt", binary=True, at=10, number_features=136, bin_cutoff=1.5, cut_zeros=True)
x_train, y_train, q_train = readData(data_path="../data/MSLR-WEB10K/Fold1/train.txt", binary=True, at=10, number_features=136, bin_cutoff=1.5, cut_zeros=True)

# Define Ranker

In [21]:
def lambda_cost(nn, y0):
    return tf.reduce_mean(tf.log(1+tf.exp(nn))-nn)


# Load directRanker, train, and test
dr = directRanker(
    feature_activation=tf.nn.tanh,
    ranking_activation=tf.nn.tanh,
    # max_steps=10000,
    # For debugging
    #cost=lambda_cost,
    max_steps=10000,
    print_step=500,
    start_batch_size=3,
    end_batch_size=5,
    start_qids=20,
    end_qids=100,
    feature_bias=True,
    hidden_layers=[100, 50, 5]
)

# Fit

In [22]:
dr.fit(x_train, y_train, ranking=True)

INFO:tensorflow:Scale of 0 disables regularizer.
step: 0, value: 1.6413570642471313, samples: 3, queries: 20
step: 500, value: 0.8485125303268433, samples: 3, queries: 21
step: 1000, value: 0.8208735585212708, samples: 3, queries: 23
step: 1500, value: 1.0172587633132935, samples: 3, queries: 25
step: 2000, value: 0.8311473727226257, samples: 3, queries: 27
step: 2500, value: 0.8592342734336853, samples: 3, queries: 29
step: 3000, value: 0.8552649021148682, samples: 3, queries: 32
step: 3500, value: 0.8993932604789734, samples: 3, queries: 35
step: 4000, value: 0.9237943887710571, samples: 3, queries: 38
step: 4500, value: 0.863290011882782, samples: 3, queries: 41
step: 5000, value: 0.8462806940078735, samples: 3, queries: 44
step: 5500, value: 0.841817319393158, samples: 3, queries: 48
step: 6000, value: 0.9386919736862183, samples: 4, queries: 52
step: 6500, value: 0.740372359752655, samples: 4, queries: 56
step: 7000, value: 0.7669996023178101, samples: 4, queries: 61
step: 7500, v

# Evaluate

In [23]:
nDCGScorer_cls(dr, x_test, y_test, at=10)
MAP_cls(dr, x_test, y_test)

nDCG@10: 0.3914 +- 0.2399
MAP: 0.3325 +- 0.1765


0.33248657967096584